In [1]:
import cv2
import numpy as np
import zipfile
from io import BytesIO

# Path to your ZIP folder containing prediction masks
zip_path = r"C:\Users\anesh\Downloads\predicted_masks.zip"  # Update this path

# Initialize coverage statistics
total_coverage = 0
image_count = 0

with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    for file_name in zip_ref.namelist():
        # Process only image files (add other extensions if needed)
        if file_name.lower().endswith(('.png', '.jpg', '.jpeg')):
            # Read image from ZIP
            with zip_ref.open(file_name) as file:
                img_data = file.read()
                nparr = np.frombuffer(img_data, np.uint8)
                prediction_mask = cv2.imdecode(nparr, cv2.IMREAD_COLOR)
                
                # Convert to RGB and create seaweed mask
                prediction_mask_rgb = cv2.cvtColor(prediction_mask, cv2.COLOR_BGR2RGB)
                seaweed_mask = np.any(prediction_mask_rgb != [0, 0, 0], axis=-1)
                
                # Calculate metrics
                total_pixels = seaweed_mask.size
                seaweed_pixels = np.sum(seaweed_mask)
                coverage = (seaweed_pixels / total_pixels) * 100
                
                # Update statistics
                total_coverage += coverage
                image_count += 1
                
                # Print individual results
                print(f"Image: {file_name}")
                print(f"- Seaweed Pixels: {seaweed_pixels}/{total_pixels}")
                print(f"- Coverage: {coverage:.2f}%\n")

# Print summary statistics
if image_count > 0:
    avg_coverage = total_coverage / image_count
    print(f"\nSummary across {image_count} images:")
    print(f"- Average Coverage: {avg_coverage:.2f}%")
else:
    print("No valid images found in the ZIP folder.")

Image: predicted_masks/predicted_mask_0.png
- Seaweed Pixels: 23473/65536
- Coverage: 35.82%

Image: predicted_masks/predicted_mask_1.png
- Seaweed Pixels: 32468/65536
- Coverage: 49.54%

Image: predicted_masks/predicted_mask_10.png
- Seaweed Pixels: 52188/65536
- Coverage: 79.63%

Image: predicted_masks/predicted_mask_11.png
- Seaweed Pixels: 56488/65536
- Coverage: 86.19%

Image: predicted_masks/predicted_mask_12.png
- Seaweed Pixels: 25310/65536
- Coverage: 38.62%

Image: predicted_masks/predicted_mask_13.png
- Seaweed Pixels: 27443/65536
- Coverage: 41.87%

Image: predicted_masks/predicted_mask_14.png
- Seaweed Pixels: 47283/65536
- Coverage: 72.15%

Image: predicted_masks/predicted_mask_15.png
- Seaweed Pixels: 25310/65536
- Coverage: 38.62%

Image: predicted_masks/predicted_mask_16.png
- Seaweed Pixels: 56677/65536
- Coverage: 86.48%

Image: predicted_masks/predicted_mask_17.png
- Seaweed Pixels: 53830/65536
- Coverage: 82.14%

Image: predicted_masks/predicted_mask_18.png
- Seawe

In [3]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import zipfile
import os
import csv

def process_zip(zip_path, output_dir):
    os.makedirs(output_dir, exist_ok=True)
    
    # Initialize metrics
    total_coverage = 0
    total_area_km2 = 0
    image_count = 0
    gsd_m2_per_pixel = 900  # 30m x 30m = 900m²/pixel
    results = []

    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
        for file_name in zip_ref.namelist():
            if file_name.lower().endswith(('.png', '.jpg', '.jpeg')):
                with zip_ref.open(file_name) as file:
                    # Process image
                    img_data = file.read()
                    nparr = np.frombuffer(img_data, np.uint8)
                    prediction_mask = cv2.imdecode(nparr, cv2.IMREAD_COLOR)
                    prediction_mask_rgb = cv2.cvtColor(prediction_mask, cv2.COLOR_BGR2RGB)
                    
                    # Calculate metrics
                    seaweed_mask = np.any(prediction_mask_rgb != [0, 0, 0], axis=-1)
                    total_pixels = seaweed_mask.size
                    seaweed_pixels = np.sum(seaweed_mask)
                    coverage = (seaweed_pixels / total_pixels) * 100
                    area_km2 = (seaweed_pixels * gsd_m2_per_pixel) / 1e6
                    
                    # Store results
                    results.append({
                        'filename': os.path.basename(file_name),
                        'coverage_pct': coverage,
                        'area_km2': area_km2
                    })
                    
                    # Update totals
                    total_coverage += coverage
                    total_area_km2 += area_km2
                    image_count += 1

                    # Create visualization
                    visualization = np.zeros_like(prediction_mask_rgb)
                    visualization[:] = [200, 200, 200]
                    visualization[seaweed_mask] = [0, 255, 0]
                    
                    # Plot and save
                    plt.figure(figsize=(12, 6))
                    plt.subplot(1, 2, 1)
                    plt.imshow(prediction_mask_rgb)
                    plt.title("Original Mask\n{}".format(os.path.basename(file_name)))
                    plt.axis("off")
                    
                    plt.subplot(1, 2, 2)
                    plt.imshow(visualization)
                    plt.title(
                        f"Seaweed Analysis\n"
                        f"Coverage: {coverage:.2f}%\n"
                        f"Area: {area_km2:.2f} km²"
                    )
                    plt.axis("off")
                    
                    output_path = os.path.join(output_dir, f"VIS_{os.path.basename(file_name)}")
                    plt.savefig(output_path, bbox_inches='tight', dpi=100)
                    plt.close()

    # Save results to CSV
    csv_path = os.path.join(output_dir, "seaweed_metrics.csv")
    with open(csv_path, 'w', newline='') as f:
        writer = csv.DictWriter(f, fieldnames=['filename', 'coverage_pct', 'area_km2'])
        writer.writeheader()
        writer.writerows(results)
        writer.writerow({
            'filename': 'TOTAL',
            'coverage_pct': total_coverage/image_count if image_count else 0,
            'area_km2': total_area_km2
        })

    # Print summary
    if image_count > 0:
        print(f"\n{' Filename ':-^50}")
        for res in results:
            print(f"{res['filename'][:40]:<40} | {res['coverage_pct']:>6.2f}% | {res['area_km2']:>9.4f} km²")
        
        print(f"\n{' Total ':-^50}")
        print(f"{'Average Coverage':<40} | {total_coverage/image_count:.2f}%")
        print(f"{'Total Seaweed Area':<40} | {total_area_km2:.4f} km²")
        print(f"\nResults saved to:\n- Visualizations: {output_dir}\n- Metrics CSV: {csv_path}")
    else:
        print("No valid images processed")

# Configuration
zip_path = r"C:\Users\anesh\Downloads\predicted_masks.zip"
output_dir = r"C:\Users\anesh\Desktop\Quantification"

# Run processing
process_zip(zip_path, output_dir)


-------------------- Filename --------------------
predicted_mask_0.png                     |  35.82% |   21.1257 km²
predicted_mask_1.png                     |  49.54% |   29.2212 km²
predicted_mask_10.png                    |  79.63% |   46.9692 km²
predicted_mask_11.png                    |  86.19% |   50.8392 km²
predicted_mask_12.png                    |  38.62% |   22.7790 km²
predicted_mask_13.png                    |  41.87% |   24.6987 km²
predicted_mask_14.png                    |  72.15% |   42.5547 km²
predicted_mask_15.png                    |  38.62% |   22.7790 km²
predicted_mask_16.png                    |  86.48% |   51.0093 km²
predicted_mask_17.png                    |  82.14% |   48.4470 km²
predicted_mask_18.png                    |  22.15% |   13.0662 km²
predicted_mask_19.png                    |  80.55% |   47.5101 km²
predicted_mask_2.png                     |  80.52% |   47.4939 km²
predicted_mask_20.png                    |  64.99% |   38.3319 km²
predicted_